# Εργασία 2 (Τεχνικές Εξόρυξης Δεδομένων)
## Data Mining: Assignment 2
***
### Μαρία Φριτζελά 1115201400218
***

In [1]:
import glob
import csv
import re
import pandas as pd
import numpy as np
from itertools import chain
from IPython.core.display import display
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import  svm, metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

Κατηγοριοποίηση δεδομένων κειμένου από ειδησιογραφικά άρθρα 5 κατηγοριών:</br>
Classifying text data from articles of 5 different categories:

In [2]:
categories = ['business', 'entertainment', 'politics', 'sport', 'tech']

### Get names of files for testing and training
Data set consists of 2225 documents from a news website
corresponding to stories in five topical areas from 2004-2005.</br>
</br>
80% of data points (files) will be used for training, the remaining 20% will be used for testing.</br>
We will be collecting the files names as elements in two lists (one for each purpose)

In [3]:
files_path = 'fulltext/data/'
train_files = []    # list of files names that will be used for training
test_files = []     # list of files names that will be used for testing

for category in categories:
    # get all txt files names from current category
    files = glob.glob(files_path+category+'/*.txt')
    # sort them alphabetically
    files.sort()
    # separate list so that first 80% will be copied into the train_files list
    sep_index = round(len(files) * 0.8)
    train_files.extend(files[:sep_index])
    test_files.extend(files[sep_index:])

Result:

In [ ]:
print("Total files: 2225")
print("# of train files: "+str(len(train_files)))
print("# of test files: "+str(len(test_files)))

### Create file train_set.tsv
Columns: id, title, content, category <br>
_The id is the name of the text file with the first letter of the category prepended (ex. "b001")_

In [ ]:
with open('train_set.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    # write header row
    tsv_writer.writerow(['id', 'title', 'content', 'category'])
    # write rows for all other files
    for file_path in train_files:
        with open(file_path) as f:
            # read all lines of file split them in a list removing '\n'
            # iterate through elements in list removing empty ones (empty strings are False)
            try:
                content = [line for line in f.read().splitlines() if line.strip()]
                # unpack the information we need from the file path
                _, _, cat, name, _ = file_path.replace('.', '/').split('/')
                # Write row
                tsv_writer.writerow([cat[0]+name,
                                 content.pop(0),
                                 " ".join(content),
                                 cat])
            except UnicodeDecodeError:
                print("UnicodeDecodeError for file: "+file_path+". File skipped")


### Create file test_set.tsv
Columns: id, title, content

In [ ]:
with open('test_set.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    # write header row
    tsv_writer.writerow(['id', 'title', 'content'])
    # write rows for all other files
    for file_path in test_files:
        with open(file_path) as f:
            # read all lines of file split them in a list removing '\n'
            # iterate through elements in list removing empty ones (empty strings are False)
            try:
                content = [line for line in f.read().splitlines() if line.strip()]
                # unpack the information we need from the file path
                _, _, cat, name, _ = file_path.replace('.', '/').split('/')
                # Write row
                tsv_writer.writerow([cat[0]+name,
                                 content.pop(0),
                                 " ".join(content)])
            except UnicodeDecodeError:
                print("UnicodeDecodeError for file: "+file_path+". File skipped")

### Create a file with the full dataset: dataset.tsv


In [4]:
with open('dataset.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    # write header row
    tsv_writer.writerow(['id', 'title', 'content', 'category'])
    # write rows for all other files
    for file_path in chain(train_files, test_files):
        with open(file_path) as f:
            # read all lines of file split them in a list removing '\n'
            # iterate through elements in list removing empty ones (empty strings are False)
            try:
                content = [line for line in f.read().splitlines() if line.strip()]
                # unpack the information we need from the file path
                _, _, cat, name, _ = file_path.replace('.', '/').split('/')
                # Write row
                tsv_writer.writerow([cat[0]+name,
                                 content.pop(0),
                                 " ".join(content),
                                 cat])
            except UnicodeDecodeError:
                print("UnicodeDecodeError for file: "+file_path+". File skipped")

UnicodeDecodeError for file: fulltext/data/sport/199.txt. File skipped


Create a DataFrame for the data_set (id column as the index)

In [5]:
#testdf = pd.read_csv("test_set.tsv", sep='\t', index_col='id')
#traindf = pd.read_csv("train_set.tsv", sep='\t', index_col='id')
datadf = pd.read_csv("dataset.tsv", sep='\t', index_col='id')
datadf

,title,content,category
id,,,
b001,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarner...,business
b002,Dollar gains on Greenspan speech,The dollar has hit its highest level against t...,business
b003,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuko...,business
b004,High fuel prices hit BA's profits,British Airways has blamed high fuel prices fo...,business
b005,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Domec...,business
...,...,...,...
t397,BT program to beat dialler scams,BT is introducing two initiatives to help beat...,tech
t398,Spam e-mails tempt net shoppers,Computer users across the world continue to ig...,tech
t399,Be careful how you code,A new European directive could put software wr...,tech


## 1 Δημιουργία WordCloud
**Create a WordCloud for the articles of each category**

In [6]:
# Create our own stopWord list:
stopwords = set(STOPWORDS)
stopwords.update(['say', 'said', 'saying', 'will', 'many', 'new', 'people', 'now', 'one'])

### Business

In [ ]:
#create wordcloud
# select rows where the id contains 'b' (=business) using filter
wordcloud_business = WordCloud(
    width = 800,
    height = 800,
    background_color = 'black',
    stopwords = stopwords).generate(" ".join(title+' '+content
                                             for title, content in datadf.filter(like='b', axis=0)
                                                                   [['title', 'content']].values.tolist()))

In [ ]:
business_image = wordcloud_business.to_image()
display(business_image)

### Entertainment

In [ ]:
wordcloud_entertainment = WordCloud(
    width = 800,
    height = 800,
    background_color = 'black',
    stopwords = stopwords).generate(str(datadf.filter(like='e', axis=0)[['title', 'content']].values))

In [ ]:
#show it
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud_entertainment)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

### Politics

In [ ]:
wordcloud_politics = WordCloud(
    width = 800,
    height = 800,
    background_color = 'black',
    stopwords = stopwords).generate(str(datadf.filter(like='p', axis=0)[['title', 'content']].values))

In [ ]:
#show it
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud_politics)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

### Sport

In [ ]:
wordcloud_sport = WordCloud(
    width = 800,
    height = 800,
    background_color = 'black',
    stopwords = stopwords).generate(str(datadf.filter(like='s', axis=0)[['title', 'content']].values))

In [ ]:
#show it
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud_sport)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

### Tech

In [ ]:
wordcloud_tech = WordCloud(
    width = 800,
    height = 800,
    background_color = 'black',
    stopwords = stopwords).generate(str(datadf.filter(like='t', axis=0)[['title', 'content']].values))

In [ ]:
#show it
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud_tech)
plt.axis("off")
plt.tight_layout(pad = 0)
plt.show()

## 2 Υλοποίηση Κατηγοριοποίησης (Classification)
**Data Classification**

### A) Cleaning and Pre-processing the data
Create a Pandas Series, adding it as a new row into the datadf,
by concatenating the title and content column of the datadf <br>
Clean up text:<br>
- Add a space before performing the sum to not connect words together accidentally
- Make all the words lower case to facilitate clean up, using `.lower`
- Remove our list of stopwords
- Remove punctuation and special characters using `re.sub`
- Remove all words containing digits, and any digits using `re.sub`.

In [7]:
datadf['text'] = datadf[['title', 'content']]\
    .apply(lambda row: ' '.join(row.values.astype(str)), axis=1)\
    .apply(lambda item: list(filter(lambda word: word not in stopwords, item.lower().split())))\
    .apply(lambda item: re.sub('[^A-Za-z0-9 ]+', '', ' '.join(item)))\
    .apply(lambda item: re.sub(r'\w*\d\w*', '', item))


#### Bag-of-words
Create bag-of-words vector

In [8]:
bow_vectorizer = CountVectorizer(max_features=3000, stop_words='english')

bow_X = bow_vectorizer.fit_transform(datadf.text)


In [9]:
pd.DataFrame(bow_X[0:1].T.todense(), index=bow_vectorizer.get_feature_names(), columns=["counts"])\
.sort_values(by=["counts"],ascending=False)

,counts
profits,5
profit,5
aol,5
sales,4
internet,4
...,...
fewer,0
fiat,0
field,0
fifth,0


#### TF-IDF

In [10]:
tfidf_vectorizer= TfidfVectorizer(ngram_range=(1,2), max_features=3000, stop_words='english')

tfidf_X = tfidf_vectorizer.fit_transform(datadf.text)

In [11]:
pd.DataFrame(tfidf_X[0:1].T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])\
.sort_values(by=["tfidf"],ascending=False)

,tfidf
profit,0.317210
profits,0.300447
fourth quarter,0.231995
internet,0.203663
stake,0.201039
...,...
filesharing,0.000000
film festival,0.000000
films,0.000000
finally,0.000000


Seperate data into train (80%) and test (20%) set <br>
Use the stratify parameter to ensure that the split between the different categories is done equally

In [12]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(bow_X, datadf.category,
                                                                test_size=0.2, stratify=datadf.category)

X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf_X, datadf.category,
                                                     test_size=0.2, stratify=datadf.category)

### B) Classification

#### Support Vector Machines (SVM)
*Bag of Words*


In [13]:
#instantiate the model
svm_clf = svm.SVC()

# train the model on the BoW training set
svm_clf.fit(X_train_bow, y_train_bow)
# predict the BoW test set
y_pred_svm_bow = svm_clf.predict(X_test_bow)

In [14]:
print("10-fold Cross Validation Precision SVM for BoW:",
      np.mean(cross_val_score(svm_clf, X_train_bow, y_train_bow, cv=10, scoring='precision_macro')))
print("10-fold Cross Validation Recall SVM for BoW:",
      np.mean(cross_val_score(svm_clf, X_train_bow, y_train_bow, cv=10, scoring='recall_macro')))
print("10-fold Cross Validation F-Measure SVM for BoW:",
     np.mean(cross_val_score(svm_clf, X_train_bow, y_train_bow, cv=10, scoring='f1_macro')))
print("10-fold Cross Validation Accuracy SVM for BoW:",
      np.mean(cross_val_score(svm_clf, X_train_bow, y_train_bow, cv=10, scoring='accuracy')))

10-fold Cross Validation Precision SVM for BoW: 0.9565175917783352
10-fold Cross Validation Recall SVM for BoW: 0.954742944556408
10-fold Cross Validation F-Measure SVM for BoW: 0.9547562765545926
10-fold Cross Validation Accuracy SVM for BoW: 0.9561480352948646


*TF/IDF*

In [15]:
# train the model on the TF/IDF training set (previous weights and variables are reset)
svm_clf.fit(X_train_tfidf, y_train_tfidf)
# predict the TF/IDF test set
y_pred_svm_tfidf = svm_clf.predict(X_test_tfidf)

In [16]:
print("10-fold Cross Validation Precision SVM for TF/IDF:",
      np.mean(cross_val_score(svm_clf, X_train_tfidf, y_train_tfidf, cv=10, scoring='precision_macro')))
print("10-fold Cross Validation Recall SVM for TF/IDF:",
      np.mean(cross_val_score(svm_clf, X_train_tfidf, y_train_tfidf, cv=10, scoring='recall_macro')))
print("10-fold Cross Validation F-Measure SVM for TF/IDF:",
     np.mean(cross_val_score(svm_clf, X_train_tfidf, y_train_tfidf, cv=10, scoring='f1_macro')))
print("10-fold Cross Validation Accuracy SVM for TF/IDF:",
      np.mean(cross_val_score(svm_clf, X_train_tfidf, y_train_tfidf, cv=10, scoring='accuracy')))

10-fold Cross Validation Precision SVM for TF/IDF: 0.9787102885785025
10-fold Cross Validation Recall SVM for TF/IDF: 0.9772926501442434
10-fold Cross Validation F-Measure SVM for TF/IDF: 0.9776337232521144
10-fold Cross Validation Accuracy SVM for TF/IDF: 0.9780803656446391


#### Random Forest

In [17]:
# Instantiate the model
rf = RandomForestClassifier()

# Train the model on the BoW training set
rf.fit(X_train_bow, y_train_bow)
# predict the BoW test set
y_pred_rf_bow = rf.predict(X_test_bow)

In [19]:
print("10-fold Cross Validation Precision RF for BoW:",
      np.mean(cross_val_score(rf, X_train_bow, y_train_bow, cv=10, scoring='precision_macro')))
print("10-fold Cross Validation Recall RF for BoW:",
      np.mean(cross_val_score(rf, X_train_bow, y_train_bow, cv=10, scoring='recall_macro')))
print("10-fold Cross Validation F-Measure RF for BoW:",
     np.mean(cross_val_score(rf, X_train_bow, y_train_bow, cv=10, scoring='f1_macro')))
print("10-fold Cross Validation Accuracy RF for BoW:",
      np.mean(cross_val_score(rf, X_train_bow, y_train_bow, cv=10, scoring='accuracy')))

10-fold Cross Validation Precision RF for BoW: 0.9584077891919411
10-fold Cross Validation Recall RF for BoW: 0.9512703472920532
10-fold Cross Validation F-Measure RF for BoW: 0.9564563907493066
10-fold Cross Validation Accuracy RF for BoW: 0.9572875007934997


In [20]:
# train the model on the TF/IDF training set (previous weights and variables are reset)
rf.fit(X_train_tfidf, y_train_tfidf)
# predict the TF/IDF test set
y_pred_rf_tfidf = rf.predict(X_test_tfidf)

In [21]:
print("10-fold Cross Validation Precision RF for TF/IDF:",
      np.mean(cross_val_score(rf, X_train_tfidf, y_train_tfidf, cv=10, scoring='precision_macro')))
print("10-fold Cross Validation Recall RF for TF/IDF:",
      np.mean(cross_val_score(rf, X_train_tfidf, y_train_tfidf, cv=10, scoring='recall_macro')))
print("10-fold Cross Validation F-Measure RF for TF/IDF:",
     np.mean(cross_val_score(rf, X_train_tfidf, y_train_tfidf, cv=10, scoring='f1_macro')))
print("10-fold Cross Validation Accuracy RF for TF/IDF:",
      np.mean(cross_val_score(rf, X_train_tfidf, y_train_tfidf, cv=10, scoring='accuracy')))

10-fold Cross Validation Precision RF for TF/IDF: 0.9559219604156663
10-fold Cross Validation Recall RF for TF/IDF: 0.9537794184264078
10-fold Cross Validation F-Measure RF for TF/IDF: 0.9556791794861443
10-fold Cross Validation Accuracy RF for TF/IDF: 0.9561639052878818


#### Naive Bayes

In [24]:
# Instantiate the model
nb = GaussianNB()

# Train the model on the BoW training set
nb.fit(X_train_bow.toarray(), y_train_bow)
# predict the BoW test set
y_pred_nb_bow = nb.predict(X_test_bow.toarray())

In [26]:
print("10-fold Cross Validation Precision NB for BoW:",
      np.mean(cross_val_score(nb, X_train_bow.toarray(), y_train_bow, cv=10, scoring='precision_macro')))
print("10-fold Cross Validation Recall NB for BoW:",
      np.mean(cross_val_score(nb, X_train_bow.toarray(), y_train_bow, cv=10, scoring='recall_macro')))
print("10-fold Cross Validation F-Measure NB for BoW:",
     np.mean(cross_val_score(nb, X_train_bow.toarray(), y_train_bow, cv=10, scoring='f1_macro')))
print("10-fold Cross Validation Accuracy NB for BoW:",
      np.mean(cross_val_score(nb, X_train_bow.toarray(), y_train_bow, cv=10, scoring='accuracy')))

10-fold Cross Validation Precision NB for BoW: 0.9117331574099212
10-fold Cross Validation Recall NB for BoW: 0.907036509632821
10-fold Cross Validation F-Measure NB for BoW: 0.9062662308179504
10-fold Cross Validation Accuracy NB for BoW: 0.9089601980575128


In [27]:
# Train the model on the TF/IDF training set (previous weights and variables are reset)
nb.fit(X_train_tfidf.toarray(), y_train_tfidf)
# predict the TF/IDF test set
y_pred_nb_tfidf = nb.predict(X_test_tfidf.toarray())

In [29]:
print("10-fold Cross Validation Precision NB for TF/IDF:",
      np.mean(cross_val_score(nb, X_train_tfidf.toarray(), y_train_tfidf, cv=10, scoring='precision_macro')))
print("10-fold Cross Validation Recall NB for TF/IDF:",
      np.mean(cross_val_score(nb, X_train_tfidf.toarray(), y_train_tfidf, cv=10, scoring='recall_macro')))
print("10-fold Cross Validation F-Measure NB for TF/IDF:",
     np.mean(cross_val_score(nb, X_train_tfidf.toarray(), y_train_tfidf, cv=10, scoring='f1_macro')))
print("10-fold Cross Validation Accuracy NB for TF/IDF:",
      np.mean(cross_val_score(nb, X_train_tfidf.toarray(), y_train_tfidf, cv=10, scoring='accuracy')))

10-fold Cross Validation Precision NB for TF/IDF: 0.9191979154110038
10-fold Cross Validation Recall NB for TF/IDF: 0.9141024978226753
10-fold Cross Validation F-Measure NB for TF/IDF: 0.9137376485180313
10-fold Cross Validation Accuracy NB for TF/IDF: 0.9168158446010283


#### K-Nearest Neighbor

Create the majority voting function. It returns the value that appears more often in List. If two values have the same number of appearances in the list, it returns the first element of the list.

In [90]:
from statistics import mode, StatisticsError

def maj_voting(List): 
    try:
        return mode(List)
    except StatisticsError:
        return List[0]

We will be creating our own KNN estimator which will be scikit-learn-compatible

In [91]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import euclidean_distances
from sklearn.neighbors import KDTree

class MyKNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_neighbors=5, leaf_size=40):
        self.n_neighbors = n_neighbors
        # leaf size for the KDTree
        self.leaf_size = leaf_size

    def fit(self, X, y):
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)

        self.X_ = X
        self.y_ = y
        # Create the KDTree
        self._tree = KDTree(X, self.leaf_size)
        
        # Return the classifier
        return self

    def predict(self, X):
        # Check is fit had been called
        check_is_fitted(self)
        
        # Input validation
        X = check_array(X)

        #closest = np.argmin(euclidean_distances(X, self.X_), axis=1)
        closest_dist, closest_indices = self._tree.query(X, k=self.n_neighbors)
        return np.array(list(map(maj_voting, self.y_[closest_indices])))


Check whether estimator adheres to the scikit-learn interface and standards

In [92]:
from sklearn.utils.estimator_checks import check_estimator
check_estimator(MyKNNClassifier)

Use our custom classifier for our dataset (BoW and TF/IDF):

In [99]:
# Instantiate the model
myclf = MyKNNClassifier(n_neighbors=2)

# Train the model on the BoW training set
myclf.fit(X_train_bow.toarray(), y_train_bow)
# predict the BoW test set
y_pred_test_bow = myclf.predict(X_test_bow.toarray())

In [100]:
print("Accuracy of My KNN for BoW:",metrics.accuracy_score(y_test_bow, y_pred_test_bow))

Accuracy of My KNN for BoW: 0.8224719101123595


#### Scores of Classifiers

In [101]:
#Model Precision: what percentage was classified correctly?
print("Precision SVM for BoW:",metrics.precision_score(y_test_bow, y_pred_svm_bow, average=None))
print("Precision SVM for TF/IDF:",metrics.precision_score(y_test_tfidf, y_pred_svm_tfidf, average=None))
print("Precision RF for BoW:",metrics.precision_score(y_test_bow, y_pred_rf_bow, average=None))
print("Precision RF for TF/IDF:",metrics.precision_score(y_test_tfidf, y_pred_rf_tfidf, average=None))
print("Precision NB for BoW:",metrics.precision_score(y_test_bow, y_pred_nb_bow, average=None))
print("Precision NB for TF/IDF:",metrics.precision_score(y_test_tfidf, y_pred_nb_tfidf, average=None))

print()
# Model Recall
print("Recall SVM for BoW:",metrics.recall_score(y_test_bow, y_pred_svm_bow, average=None))
print("Recall SVM for TF/IDF:",metrics.recall_score(y_test_tfidf, y_pred_svm_tfidf, average=None))
print("Recall RF for BoW:",metrics.recall_score(y_test_bow, y_pred_rf_bow, average=None))
print("Recall RF for TF/IDF:",metrics.recall_score(y_test_tfidf, y_pred_rf_tfidf, average=None))
print("Recall NB for BoW:",metrics.recall_score(y_test_bow, y_pred_nb_bow, average=None))
print("Recall NB for TF/IDF:",metrics.recall_score(y_test_tfidf, y_pred_nb_tfidf, average=None))

print()
# F-Measure
print("F-Measure SVM for BoW:", metrics.f1_score(y_test_bow, y_pred_svm_bow, average=None))
print("F-Measure SVM for TF/IDF:", metrics.f1_score(y_test_tfidf, y_pred_svm_tfidf, average=None))
print("F-Measure RF for BoW:", metrics.f1_score(y_test_bow, y_pred_rf_bow, average=None))
print("F-Measure RF for TF/IDF:", metrics.f1_score(y_test_tfidf, y_pred_rf_tfidf, average=None))
print("F-Measure NB for BoW:", metrics.f1_score(y_test_bow, y_pred_nb_bow, average=None))
print("F-Measure NB for TF/IDF:", metrics.f1_score(y_test_tfidf, y_pred_nb_tfidf, average=None))

print()
# compare actual response values (y_test) with predicted response values (y_pred)
print("Accuracy SVM for BoW:",metrics.accuracy_score(y_test_bow, y_pred_svm_bow))
print("Accuracy SVM for TF/IDF:",metrics.accuracy_score(y_test_tfidf, y_pred_svm_tfidf))
print("Accuracy RF for BoW:",metrics.accuracy_score(y_test_bow, y_pred_rf_bow))
print("Accuracy RF for TF/IDF:",metrics.accuracy_score(y_test_tfidf, y_pred_rf_tfidf))
print("Accuracy NB for BoW:",metrics.accuracy_score(y_test_bow, y_pred_nb_bow))
print("Accuracy NB for TF/IDF:",metrics.accuracy_score(y_test_tfidf, y_pred_nb_tfidf))

Precision SVM for BoW: [0.98947368 0.98701299 0.96428571 0.98019802 0.875     ]
Precision SVM for TF/IDF: [0.98969072 1.         0.95402299 1.         0.95238095]
Precision RF for BoW: [0.97       0.97402597 0.97590361 0.95327103 0.94871795]
Precision RF for TF/IDF: [0.93333333 0.97333333 0.97530864 0.97142857 0.93670886]
Precision NB for BoW: [0.94565217 0.92857143 0.89655172 0.98989899 0.79381443]
Precision NB for TF/IDF: [0.91208791 0.92537313 0.87209302 1.         0.74509804]

Recall SVM for BoW: [0.92156863 0.98701299 0.96428571 0.97058824 0.9625    ]
Recall SVM for TF/IDF: [0.94117647 0.97402597 0.98809524 1.         1.        ]
Recall RF for BoW: [0.95098039 0.97402597 0.96428571 1.         0.925     ]
Recall RF for TF/IDF: [0.96078431 0.94805195 0.94047619 1.         0.925     ]
Recall NB for BoW: [0.85294118 0.84415584 0.92857143 0.96078431 0.9625    ]
Recall NB for TF/IDF: [0.81372549 0.80519481 0.89285714 0.97058824 0.95      ]

F-Measure SVM for BoW: [0.95431472 0.98701299 